In [4]:
import numpy as np
import onnxruntime as ort
from PIL import Image
from scipy.spatial.distance import cosine
import pickle

def preprocess_image(image_path, input_size=(224, 224)):
    image = Image.open(image_path)
    image = image.convert('RGB')
    image = image.resize(input_size)
    image_array = np.array(image)
    image_array = image_array / 255.0
    image_array = image_array.astype(np.float32)
    image_array = np.transpose(image_array[np.newaxis, ...], (0,3,1,2))
    return image_array

def run_inference(image_path, model_path):
    session = ort.InferenceSession(model_path)
    input_name = session.get_inputs()[0].name
    input_data = preprocess_image(image_path)
    outputs = session.run(None, {input_name: input_data})
    probabilities = softmax(outputs[0][0])
    predicted_class = np.argmax(probabilities)
    return predicted_class, probabilities, outputs[1]

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def load_mean_embedding(file_path):
    """평균 임베딩 로드 함수

    Args :
        - file_path : pickle file path
    """

    with open(file_path, "rb") as f:
        data = pickle.load(f)
        print(data.shape)

        return data


def is_eye_image(extract_embedding, mean_embedding, threshold=0.65):
    # Flatten the embeddings to 1-D arrays
    extract_embedding_flat = extract_embedding.squeeze().flatten()
    mean_embedding_flat = mean_embedding.flatten()
    print(extract_embedding_flat.shape, mean_embedding_flat.shape)
    similarity = 1 - cosine(extract_embedding_flat, mean_embedding_flat)
    result = similarity > threshold

    return result, similarity



In [5]:
import os

# 이미지가 있는 기본 디렉토리 경로
base_dir = "/workspace/a-eye-lab-research/export/test_data"
model_path = "/workspace/model_quantized.onnx"

mean_emb = load_mean_embedding("/workspace/a-eye-lab-research/notebook/eye_detection/embedding_pkl/mean_embedding_tuning_250118.pkl")
# 모든 하위 디렉토리(0, 1)를 순회
for class_dir in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_dir)
    if os.path.isdir(class_path):
        print(f"\nProcessing class directory: {class_dir}")
        
        # 각 클래스 디렉토리 내의 모든 이미지 파일을 순회
        for image_file in os.listdir(class_path):
            if image_file.endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(class_path, image_file)
                
                # 추론 실행
                predicted_class, probabilities, embedding = run_inference(image_path, model_path)


                print("=== 이미지 유사성 판단 ===")

                result, similarity = is_eye_image(
                    embedding, mean_emb, threshold=0.65
                )

                print(
                    f"눈 이미지, {np.round(similarity,2)}"
                    if result
                    else f"눈 이미지 아님, {np.round(similarity,2)}"
                )
                
                # 결과 출력
                print(f"\nImage: {image_file}")
                print(f"True class: {class_dir}")
                print(f"Predicted class: {predicted_class}")
                print(f"Probabilities: [{probabilities[0]:.4f}, {probabilities[1]:.4f}]")

(960,)

Processing class directory: 0
=== 이미지 유사성 판단 ===
(47040,) (960,)


2025-01-18 10:03:42.874166835 [W:onnxruntime:, graph.cc:1348 Graph] Initializer model.bn1.weight appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2025-01-18 10:03:42.874255220 [W:onnxruntime:, graph.cc:1348 Graph] Initializer model.bn1.bias appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2025-01-18 10:03:42.874263677 [W:onnxruntime:, graph.cc:1348 Graph] Initializer model.bn1.running_mean appears

ValueError: shapes (47040,) and (960,) not aligned: 47040 (dim 0) != 960 (dim 0)